In [1]:
import os

In [2]:
%pwd
os.chdir("..")

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\End-to-End-ML-project-MLflow'

In [11]:
#entity

from dataclasses import *
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    trained_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    random_state: int
    target_column: str

In [12]:
from my_project.constants import *
from my_project.utils.common import read_yaml, create_directories

# configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH,
    ):
        # Load configs
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)  # dict (safe for schema)

        # Ensure artifacts root exists
        create_directories([Path(self.config.artifacts_root)])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.target_column

        create_directories([Path(config.root_dir)])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            trained_data_path=Path(config.trained_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            random_state=params.random_state,
            target_column=schema
        )

        return model_trainer_config

In [13]:
# components
import pandas as pd
import os
from my_project import logging
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self, x_train, y_train):
        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=self.config.random_state,
        )
        model.fit(x_train, y_train)
        return model

    def initiate_model_trainer(self):
        logging.info("Loading training and test data")
        train_df = pd.read_csv(self.config.trained_data_path)
        test_df = pd.read_csv(self.config.test_data_path)

        target_column = self.config.target_column

        logging.info("Splitting training and test data into features and target")
        x_train = train_df.drop(columns=[target_column], axis=1)
        y_train = train_df[target_column]

        x_test = test_df.drop(columns=[target_column], axis=1)
        y_test = test_df[target_column]

        logging.info("Training the model")
        model = self.train_model(x_train, y_train)

        logging.info("Saving the trained model")
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

        logging.info("Model training completed")

        return model


In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.initiate_model_trainer()
except Exception as e:
    raise e

2025-09-01 12:29:17,226 - my_project - INFO - YAML file 'config\config.yaml' read successfully.
2025-09-01 12:29:17,233 - my_project - INFO - YAML file 'params.yaml' read successfully.
2025-09-01 12:29:17,235 - my_project - INFO - YAML file 'schema.yaml' read successfully.
2025-09-01 12:29:17,243 - my_project - INFO - Directory created: 'artifacts'
2025-09-01 12:29:17,245 - my_project - INFO - Directory created: 'artifacts\model_trainer'
2025-09-01 12:29:17,245 - root - INFO - Loading training and test data
2025-09-01 12:29:17,334 - root - INFO - Splitting training and test data into features and target
2025-09-01 12:29:17,340 - root - INFO - Training the model
2025-09-01 12:29:17,354 - root - INFO - Saving the trained model
2025-09-01 12:29:17,360 - root - INFO - Model training completed
